# BeautyfulSoup 네이버 블로그 스크래핑

## 블로그 포스트 스크래핑 함수 정의

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import pandas as pd

def get_posts(query, page_num=10):

  url_query = quote(query)
  url = "https://search.naver.com/search.naver?where=post&query=" + url_query
  post_df = pd.DataFrame(columns=("Title", "Date", "Blogger", "Blog URL", "Post URL", "Post"))
  idx = 0

  for _ in range(0,page_num):

    search_url = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(search_url, 'html.parser')
    posts = soup.find_all('li', {'class':'sh_blog_top'})

    for post in posts:
      title = post.find('a', {'class':'sh_blog_title _sp_each_url _sp_each_title'}).get('title')
      date = post.find('dd', {'class':'txt_inline'}).get_text()
      blogger = post.find('a', {'class':'txt84'}).get_text()
      blog_url = post.find('a', {'class':'txt84'}).get('href')
      post_url = post.find('a', {'class':'url'}).get('href')
      post_link = urllib.request.urlopen(post_url).read()
      post_html = BeautifulSoup(post_link, 'html.parser')

      for main_frame in post_html.select("iframe#mainFrame"):
        frame_url = "https://blog.naver.com" + main_frame.get('src')
        post_text = urllib.request.urlopen(frame_url).read()
        post_html = BeautifulSoup(post_text, 'html.parser')
        post_content_text = ""
        
        for post_content in post_html.find_all('div', {'class':'se-main-container'}):
          post_content_text = post_content.get_text()

        post_content_text = post_content_text.replace("\n", "")
        post_content_text = post_content_text.replace("\t", "")

        post_df.loc[idx] = [title, date, blogger, blog_url, post_url, post_content_text]
        idx += 1
        print("#", end="")
    try:
      next = soup.find('a', {'class':'next'}).get('href')
      url = "https://search.naver.com/search.naver" + next
    except:
      break

  return post_df

## 블로그 포스트 스크래핑 실행

In [ ]:
query = input("검색질의: ")
post_df = get_posts(query, 10)
print("Done")

검색질의: 컴퓨터
##############################################################################################Done


In [ ]:
post_df

,Title,Date,Blogger,Blog URL,Post URL,Post
0,2020최신기준 피시방 컴퓨터 사양,2020.08.02.,원트컴 (구.마블컴),https://blog.naver.com/all_in_box,https://blog.naver.com/all_in_box?Redirect=Log...,안녕하세요 ^^ 한주를 마무리하는 일요일입니다. 요즘 코로나로 인해서 집에서 게이밍...
1,컴퓨터 화면 캡쳐 기본 프로그램과 단축키 이용하는 방법,2020.04.16.,모이모이의 IT 이야...,https://blog.naver.com/moimoi1357,https://blog.naver.com/moimoi1357?Redirect=Log...,다양한 최신 IT 기기에 대해 살펴보고 성능과 장단점에 대한 테스트를 진행하면서 사...
2,4K 작업도 문제없는 영상편집 컴퓨터 사양,2020.08.02.,조인스컴 공식블로그,https://blog.naver.com/hanwoo10303,https://blog.naver.com/hanwoo10303?Redirect=Lo...,​오늘은 프리미어 프로와 애프터 이펙트를 사용하기 위한 영상편집 컴퓨터를 소개합니다...
3,인텔 10세대 i7 10700이 장착된 최신 사양 주식 컴퓨터,2020.07.31.,비컴의 컴퓨터이야기,https://blog.naver.com/becom5f,https://blog.naver.com/becom5f?Redirect=Log&lo...,Overview i7 10700 CPU를 장착한 최신 사양 주식 컴퓨터로 끊김 없...
4,활용도 높은 거실 컴퓨터 멋진 디자인,2일 전,애플매니아 피씨와맥,https://blog.naver.com/cutiedahee,https://blog.naver.com/cutiedahee?Redirect=Log...,편집디자인용으로 사용할 애플 컴퓨터 설치했습니다.며칠전 신형 아이맥이 발표되었는데 ...
...,...,...,...,...,...,...
89,또한번 휴대용 게이밍노트북의 신화를!! 인텔 i7 10세대 + GTX1650Ti 탑...,2020.07.09.,한성컴퓨터 공식 블로그,https://blog.naver.com/monsterlabs,https://blog.naver.com/monsterlabs?Redirect=Lo...,"한성컴퓨터,10세대 인텔 CPU로 즐기는아방가르드한 노트북라이프!​TFG5075G ..."
90,10700+RTX2070S탑재 게이밍 컴퓨터,2020.06.18.,양컴 양실장의 조립...,https://blog.naver.com/combox0,https://blog.naver.com/combox0?Redirect=Log&lo...,"게이밍 조립컴퓨터​안녕하세요, 양컴입니다.오늘은 160만원대의 게이밍 시스템 소개를..."
91,"QHD IPS 패널, 144Hz 주사율을 가진 32인치 광시야각 게이밍모니터!! 한...",2020.07.29.,한성컴퓨터 공식 블로그,https://blog.naver.com/monsterlabs,https://blog.naver.com/monsterlabs?Redirect=Lo...,한성컴퓨터 TFG32Q14P IPS QHD 144 게이밍 모니터 QHD IPS 패...
92,영화감상용 슬림 조립컴퓨터,2020.06.30.,용산 세븐컴 씨즌2 ...,https://blog.naver.com/seven8805,https://blog.naver.com/seven8805?Redirect=Log&...,안녕하세요~용산 조립컴퓨터 전문매장 세븐컴피씨 입니다.​I3-9100 + 8GB ...
